In [ ]:
!pip install tensorflow==2.13

In [ ]:
import tensorflow
import os
print(tensorflow.__version__)

In [ ]:
!pip install tflite-support

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

# Model Info
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "SeaFest Freshness"
model_meta.description = ("Identify Fish Freshness (Fresh or Not Fresh)")
model_meta.version = "1.0.1"
model_meta.author = "SeaFest | Hx3"
model_meta.license = ("SeaFest")

# Input Info
input_meta = _metadata_fb.TensorMetadataT()

# Output Info
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities Fresh or Not Fresh"
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)

output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename("FreshnessLabels.txt")
label_file.description = "Labels for Objects that Model Recognized"
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

# Subgraph Info
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)

b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()
model_file = "FreshnessModel.tflite"
populator = _metadata.MetadataPopulator.with_model_file(model_file)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files(["FreshnessLabels.txt"])
populator.populate()

In [ ]:
from google.colab import files

# Replace 'path_to_your_file.json' with the actual path of your generated JSON file
files.download('FreshnessModel.tflite')
files.download('FreshnessModel.tflite')


INSPECT POPULATED METADATA (OPTIONAL)

In [ ]:
tf.compat.v1.flags.DEFINE_string('f','/','')

In [ ]:
import absl.flags
import os
import sys

# Parse command-line flags
absl.flags.FLAGS(sys.argv)  # Parse all command-line arguments

# Define command-line flags only if not already defined
if not hasattr(absl.flags.FLAGS, 'export_directory'):
    absl.flags.DEFINE_string('export_directory', '/', 'Directory to export files')

# Assuming `model_file` and `model_basename` are defined somewhere in your code
displayer = _metadata.MetadataDisplayer.with_model_file(model_file)

# Access export directory from FLAGS or use default if not provided
export_directory = absl.flags.FLAGS.export_directory if hasattr(absl.flags.FLAGS, 'export_directory') else '/'

model_basename = "FreshnessWithMetadata"
export_json_file = os.path.join(export_directory, os.path.splitext(model_basename)[0] + ".json")
os.makedirs(os.path.dirname(export_json_file), exist_ok=True)

json_file = displayer.get_metadata_json()

# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
    f.write(json_file)

print(json_file)

In [ ]:
from tflite_support import metadata as _metadata

# Load Model
model_file_path = 'FreshnessModel.tflite' 
displayer = _metadata.MetadataDisplayer.with_model_file(model_file_path)

# Display Metadata
metadata_json = displayer.get_metadata_json()
print(metadata_json)
